In [6]:
import os

In [7]:
### replace with necessary path to val dir
SOURCE_DIR = "../../runs_val/active/near_target/"
assert os.path.isdir(SOURCE_DIR)

CKPT_DIR = os.path.join(SOURCE_DIR, "data")
assert os.path.isdir(CKPT_DIR)

TRAIN_LOG_PATH = os.path.join(SOURCE_DIR, "train.log")
assert os.path.exists(TRAIN_LOG_PATH)

In [8]:
"""mono and monoFromMem losses"""
ckpt2mono_allSteps_dct = {}
ckpt2monoFromMem_allSteps_dct = {}
ckpt_number = -1
look_for_newCkpt = True
with open(TRAIN_LOG_PATH, "r") as fo:
    for line in fo:
        if look_for_newCkpt and (len(line.split(" ")) >= 3) and (line.split(" ")[2] == "Mono") and (line.split(" ")[7] == "all") and (line.split(" ")[9] == "---"):
            ckpt2mono_allSteps_dct[ckpt_number] = [float(line.split(" ")[11][:-1]), float(line.split(" ")[13][:-1])]
    
        if look_for_newCkpt and (len(line.split(" ")) >= 3) and (line.split(" ")[2] == "MonoFromMem") and (line.split(" ")[7] == "all") and (line.split(" ")[9] == "---"):
            look_for_newCkpt = False
            ckpt2monoFromMem_allSteps_dct[ckpt_number] = [float(line.split(" ")[11][:-1]), float(line.split(" ")[13][:-1])]
                
        if (not look_for_newCkpt) and (len(line.split(" ")) >= 4) and (line.split(" ")[2] == "=======current_ckpt:"):
            look_for_newCkpt = True
            ckpt_number = int(line.split(" ")[-1].split("=======")[0].split("/")[-1].split(".")[1])            

In [9]:
ckpt_numbers = []
missing_key = -1
for (dirpath, dirnames, filenames) in os.walk(CKPT_DIR):
    ckpt_numbers.extend(filenames)
    break
    
for i in range(len(ckpt_numbers)):
    ckpt_numbers[i] = int(ckpt_numbers[i].split(".")[1])
    
for key in ckpt_numbers:
    if key not in list(ckpt2mono_allSteps_dct.keys()):
        missing_key = key
        
ckpt2mono_allSteps_dct_final = {}
ckpt2monoFromMem_allSteps_dct_final = {}
for key in ckpt2mono_allSteps_dct:
    if key == -1:
        ckpt2mono_allSteps_dct_final[missing_key] = ckpt2mono_allSteps_dct[key]
        ckpt2monoFromMem_allSteps_dct_final[missing_key] = ckpt2monoFromMem_allSteps_dct[key]
    else:
        ckpt2mono_allSteps_dct_final[key] = ckpt2mono_allSteps_dct[key]
        ckpt2monoFromMem_allSteps_dct_final[key] = ckpt2monoFromMem_allSteps_dct[key]

In [10]:
min_mono_allSteps = [float("inf"), 0]
min_monoFromMem_allSteps = [float("inf"), 0]

for key, value in ckpt2mono_allSteps_dct_final.items():
    if value[0] < min_mono_allSteps[0]:
        min_mono_allSteps = value
        bestCkpt_mono_allSteps = "ckpt." + str(key) + ".pth"
        
    if ckpt2monoFromMem_allSteps_dct_final[key][0] < min_monoFromMem_allSteps[0]:
        min_monoFromMem_allSteps = ckpt2monoFromMem_allSteps_dct_final[key]
        bestCkpt_monoFromMem_allSteps = "ckpt." + str(key) + ".pth"  
        
print("best validation checkpoint: ", bestCkpt_mono_allSteps,
      ", mono_allSteps: mean -- {}, std -- {}".format(min_mono_allSteps[0],
                                                      min_mono_allSteps[1]))
print("best validation checkpoint: ", bestCkpt_monoFromMem_allSteps,
      ", monoFromMem_allSteps: mean -- {}, std -- {}".format(min_monoFromMem_allSteps[0],
                                                             min_monoFromMem_allSteps[1]))

best validation checkpoint:  ckpt.130.pth , mono_allSteps: mean -- 0.163263, std -- 0.023723
best validation checkpoint:  ckpt.130.pth , monoFromMem_allSteps: mean -- 0.154667, std -- 0.028789
